In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [15]:
# Read in Loans dataset
# Set display to show all columns

pd.set_option("display.max_columns", None)

df = pd.read_csv("../data/SBAnational.csv")
df

c:\users\user\.virtualenvs\build_week_2-2dp-f4_g\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


LoanNr_ChkDgt                            Name             City State  \
0          1000014003                  ABC HOBBYCRAFT       EVANSVILLE    IN   
1          1000024006     LANDMARK BAR & GRILLE (THE)        NEW PARIS    IN   
2          1000034009           WHITLOCK DDS, TODD M.      BLOOMINGTON    IN   
3          1000044001   BIG BUCKS PAWN & JEWELRY, LLC     BROKEN ARROW    OK   
4          1000054004     ANASTASIA CONFECTIONS, INC.          ORLANDO    FL   
...               ...                             ...              ...   ...   
899159     9995573004                    FABRIC FARMS  UPPER ARLINGTON    OH   
899160     9995603000                    FABRIC FARMS         COLUMBUS    OH   
899161     9995613003    RADCO MANUFACTURING CO.,INC.      SANTA MARIA    CA   
899162     9995973006           MARUTAMA HAWAII, INC.         HONOLULU    HI   
899163     9996003010  PACIFIC TRADEWINDS FAN & LIGHT           KAILUA    HI   

          Zip                            Bank BankState   NAICS ApprovalDate  \
0       47711                FIFTH THIRD BANK        OH  451120    28-Feb-97   
1       46526                 1ST SOURCE BANK        IN  722410    28-Feb-97   
2       47401         GRANT COUNTY STATE BANK        IN  621210    28-Feb-97   
3       74012   1ST NATL BK & TR CO OF BROKEN        OK       0    28-Feb-97   
4       32801         FLORIDA BUS. DEVEL CORP        FL       0    28-Feb-97   
...       ...                             ...       ...     ...          ...   
899159  43221  JPMORGAN CHASE BANK NATL ASSOC        IL  451120    27-Feb-97   
899160  43221  JPMORGAN CHASE BANK NATL ASSOC        IL  451130    27-Feb-97   
899161  93455  RABOBANK, NATIONAL ASSOCIATION        CA  332321    27-Feb-97   
899162  96830                  BANK OF HAWAII        HI       0    27-Feb-97   
899163  96734            CENTRAL PACIFIC BANK        HI       0    27-Feb-97   

       ApprovalFY  Term  NoEmp  NewExist  CreateJob  RetainedJob  \
0            1997    84      4       2.0          0            0   
1            1997    60      2       2.0          0            0   
2            1997   180      7       1.0          0            0   
3            1997    60      2       1.0          0            0   
4            1997   240     14       1.0          7            7   
...           ...   ...    ...       ...        ...          ...   
899159       1997    60      6       1.0          0            0   
899160       1997    60      6       1.0          0            0   
899161       1997   108     26       1.0          0            0   
899162       1997    60      6       1.0          0            0   
899163       1997    48      1       2.0          0            0   

        FranchiseCode  UrbanRural RevLineCr LowDoc ChgOffDate  \
0                   1           0         N      Y        NaN   
1                   1           0         N      Y        NaN   
2                   1           0         N      N        NaN   
3                   1           0         N      Y        NaN   
4                   1           0         N      N        NaN   
...               ...         ...       ...    ...        ...   
899159              1           0         0      N        NaN   
899160              1           0         Y      N        NaN   
899161              1           0         N      N        NaN   
899162              1           0         N      Y   8-Mar-00   
899163              1           0         N      N        NaN   

       DisbursementDate DisbursementGross BalanceGross MIS_Status  \
0             28-Feb-99       $60,000.00        $0.00       P I F   
1             31-May-97       $40,000.00        $0.00       P I F   
2             31-Dec-97      $287,000.00        $0.00       P I F   
3             30-Jun-97       $35,000.00        $0.00       P I F   
4             14-May-97      $229,000.00        $0.00       P I F   
...                 ...               ...          ...        ...   
89915

I can see there are some zeros under NAICS so I have to check for null values before wrangling the zeros as NaNs

In [16]:
df.isnull().sum()

LoanNr_ChkDgt             0
Name                     14
City                     30
State                    14
Zip                       0
Bank                   1559
BankState              1566
NAICS                     0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist                136
CreateJob                 0
RetainedJob               0
FranchiseCode             0
UrbanRural                0
RevLineCr              4528
LowDoc                 2582
ChgOffDate           736465
DisbursementDate       2368
DisbursementGross         0
BalanceGross              0
MIS_Status             1997
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
dtype: int64

The *ChgOffDate* column is the date that the loan gets "charged off" to the SBA (Defaulting). I'll drop this column to prevent leakage

In [18]:
df = df.drop("ChgOffDate", axis=1)

The first idea for feature engineering I had was to check if the borrower's state is the same as the bank's. I'll have to drop those rows of NaNs in *BankState* to make that possible. 1,566 rows looks like a lot but I can live with that since I have 899,000 observations.

In [19]:
# Drop Bank state null rows
df = df[df["BankState"].notna()]

*MIS_Status* is the target variable so I have to drop the rows that are null as well

In [20]:
# Drop target variable with no values

df = df[df["MIS_Status"].notna()]

In [21]:
# Check number of observations and update NaN count

print(len(df))
df.isnull().sum()

895654


LoanNr_ChkDgt           0
Name                   14
City                   28
State                  13
Zip                     0
Bank                    0
BankState               0
NAICS                   0
ApprovalDate            0
ApprovalFY              0
Term                    0
NoEmp                   0
NewExist              134
CreateJob               0
RetainedJob             0
FranchiseCode           0
UrbanRural              0
RevLineCr            4511
LowDoc               2578
DisbursementDate     2173
DisbursementGross       0
BalanceGross            0
MIS_Status              0
ChgOffPrinGr            0
GrAppv                  0
SBA_Appv                0
dtype: int64

In [22]:
df[df["City"].isnull()]

LoanNr_ChkDgt                            Name City State    Zip  \
42535      1330603010              BUSATH PHOTOGRAPHY  NaN    UT  84109   
177891     2268732010              HAYES/DOCKSIDE,INC  NaN    LA  70130   
243450     2716822007       YACHTING &NAVIGATION, LTD  NaN    IL  60624   
270263     2884182004         HERMANOS LOPEZ,INC. "B"  NaN    CA  92102   
270265     2884192007  RODIECK PLUMBING SUPPLY,INC "A  NaN    CA  92102   
270268     2884202008  SAN DIEGO REFRIGERATD SVCS,INC  NaN    CA  92102   
270270     2884212000                 EMACO, INC. "B"  NaN    CA  92102   
270277     2884232006        MAYER BLUEPRINT,INC. "B"  NaN    CA  92102   
270284     2884252001                TEX-WIPE COMPANY  NaN    CA  92102   
326169     3258222000  LAKEWOOD HEALTH CARE CENTER LT  NaN    OH  44115   
342177     3386722002                 LEE'S PAWN SHOP  NaN    MO  63103   
342185     3386762003   OLIVE LIGHT LOAN & MERCANTILE  NaN    MO  63103   
342928     3391402005         MASTER PLEATING COMPANY  NaN    MO  63103   
342938     3391452009   1215 PINE COMPANY PARTNERSHIP  NaN    MO  63103   
342946     3391472004              CLARK TIRE COMPANY  NaN    MO  63105   
342951     3391492010                BRAZNELL COMPANY  NaN    MO  63013   
344784     3403402003          WORLD TIRE CORPORATION  NaN    WA  98104   
350109     3438122002                 DULTMEIER SALES  NaN    NE  50309   
378899     3662512006  AMERICAN MANAGEMENT ENG ASS IN  NaN    TX  78204   
410929     3970852005           REVERSAL SYSTEMS,INC.  NaN    MO  64106   
410933     3970882003             DOVER GRAPHICS,INC.  NaN    MO  64106   
437782     4247422000      B. G. WICKBERG COMPANY INC  NaN    MA   2169   
437788     4247472004         R & F MICRO TOOL CO INC  NaN    MA   2169   
437793     4247502000                 J SAX & COMPANY  NaN    MA   2108   
437818     4247702001        GEMCO NARROW FABRICS INC  NaN    MA   2165   
564191     5554512008           ACTION GRAPHICS, INC.  NaN    KY  40209   
570342     5622822010                  HMC PROPERTIES  NaN    NC  28402   
745271     7756831000                 B & B ASSOIATES  NaN    MI  48502   

                                  Bank BankState  NAICS ApprovalDate  \
42535     MOUNTAIN W. SMALL BUS. FINAN        UT      0    15-Dec-81   
177891  NEW ORLEANS REGIONAL BUS. DEVE        LA      0    17-Dec-81   
243450       BANK - AMERICA NATL ASSOC        IL      0    10-Apr-81   
270263       CDC SMALL BUS. FINAN CORP        CA      0    16-Apr-81   
270265       CDC SMALL BUS. FINAN CORP        CA      0    15-Jun-81   
270268       CDC SMALL BUS. FINAN CORP        CA      0    26-Jun-81   
270270       CDC SMALL BUS. FINAN CORP        CA      0    13-Jul-81   
270277       CDC SMALL BUS. FINAN CORP        CA      0    29-Jul-81   
270284       CDC SMALL BUS. FINAN CORP        CA      0    31-Aug-81   
326169            GROWTH CAPITAL CORP.        OH      0    18-Nov-81   
342177        ST. LOUIS LOCAL DEVEL CO        MO      0    16-Oct-81   
342185        ST. LOUIS LOCAL DEVEL CO        MO      0    30-Nov-81   
342928        ST. LOUIS LOCAL DEVEL CO        MO      0    21-Jul-81   
342938        ST. LOUIS LOCAL DEVEL CO        MO      0    11-May-81   
342946  BUSINESS FINAN CORP OF ST. LOU        MO      0     4-Mar-81   
342951        ST. LOUIS LOCAL DEVEL CO        MO      0    24-Feb-81   
344784      EVERGREEN BUSINESS CAPITAL        WA      0     8-Jun-81   
350109    IOWA BUSINESS GROWTH COMPANY        IA      0     7-Oct-81   
378899              ACCION TEXAS, INC.        TX      0     2-Jun-81   
410929            EDC LOAN CORPORATION        MO      0    14-Jan-81   
410933            EDC LOAN CORPORATION        MO      0    12-Mar-81   
437782  SOUTH SHORE ECONOMIC DEVEL COR        MA      0    30-Jan-81   
437788  SOUTH SHORE ECONOMIC DEVEL COR        MA      0    17-Mar-81   
437793        BOSTON LOCAL DEVELOPMENT        MA      0     1-Apr-81   
437818           BAY COLONY DEVE

I was hoping the State null rows were in the same as the City null rows so I could find them at once. Looks like I'll have to research each column one at a time from the zipcode

I stumbled upon a brilliant package that can update City and State from zip codes. Check out [*uszipcode*](https://pypi.org/project/uszipcode/0.1.2/)

In [23]:
# I tried ZipcodeSearchEngine like in the documentation but got an import error
# Autofill had a SearchEngine class so try that
from uszipcode import SearchEngine
search = SearchEngine()

In [ ]:
# Assign null city df subset to a smaller df
df_City_null = df[df["City"].isnull()]

#Use zipcode search
df_City_null["City"] = search.by_zipcode(df_City_null["Zip"])["City"]